In [321]:
import pandas as pd
import glob


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from pathlib import Path

In [322]:
input_files = str(Path('../4_szures/data/*.csv'))

In [323]:
x = pd.read_csv(glob.glob(input_files)[0]).drop(["Unnamed: 0","Unnamed: 0.1"],axis=1)\
.pipe(lambda df:df.loc[df["sold"]==1]).fillna(0).rename(columns = {"TELJESITMENY LOERO":"LOERO","MEGTETT":"MEGTETT_KM","MEGTETT_2":"MEGTETT_KM_2","BI-XENON":"BI_XENON"})

In [324]:
x = x.loc[x["vegso_ar"]!=0]


In [325]:
y = x["vegso_ar"]
lg10_y = pd.np.log10(y)
x["lg10_AR"] = lg10_y

x = x.drop(['vegso_ar',"lg10_AR"], axis=1)


In [326]:
szukitett = x[["hany_eves","hany_eves_2","MEGTETT_KM","MEGTETT_KM_2","KLÍMA","LÉGZSÁK",\
               "ALUFELNI","ASR","AUTOMATA","BI_XENON","BLUETOOTH","LOERO",\
               "audi","mercedes-benz","bmw","infiniti","jaguar","land_rover","porsche",\
               "lexus","maserati","Hibrid","Elektromos","Dízel","Benzin"]].fillna(0)

In [327]:
x_train, x_test, y_train, y_test = train_test_split(szukitett.dropna(), lg10_y, test_size=0.2, random_state=42)

lreg = LinearRegression()

result = lreg.fit(x_train, y_train)

y_test_pred = result.predict(x_test)

lin_mse = mean_squared_error(y_test, y_test_pred)
lin_rmse = pd.np.sqrt(lin_mse)
lin_rmse

print(result.intercept_)


6.587655503948437


In [328]:
pd.DataFrame(szukitett.loc[9]).to_csv("proba.csv")

In [329]:
import pickle


output_file = str(Path('data/finalized_model.sav'))

# save the model to disk

pickle.dump(result, open(output_file, 'wb'))

In [330]:
import json

In [331]:
szukitett["hany_eves"].nunique()

41

In [332]:
nuniques = [szukitett[col].nunique() for col in szukitett.columns]

In [333]:
number_vals = [idx for idx,val in enumerate(nuniques) if val >2]

In [334]:
auto_marka = ['audi', 'mercedes-benz', 'bmw', 'infiniti',
              'jaguar', 'land_rover', 'porsche', 'lexus', 'maserati']
hajtaslanc = ['Hibrid', 'Elektromos', 'Dízel', 'Benzin']
felszereltseg = szukitett.columns.drop(auto_marka + hajtaslanc + list(szukitett.columns[number_vals]))

In [335]:
felszereltseg

Index(['KLÍMA', 'LÉGZSÁK', 'ALUFELNI', 'ASR', 'AUTOMATA', 'BI_XENON',
       'BLUETOOTH'],
      dtype='object')

In [336]:
details = {"number":list(szukitett.columns[number_vals]),"Dropdown":{"auto_marka":auto_marka,"hajtaslanc":hajtaslanc}\
          ,"boolen":list(felszereltseg),"all":list(szukitett.columns)}

In [337]:
json_file = json.dumps(details)
f = open("data/dict.json","w")
f.write(json_file)
f.close()

In [338]:
with open('data/dict.json') as json_file:  
    data_dict = json.load(json_file)

In [339]:
data_dict

{'Dropdown': {'auto_marka': ['audi',
   'mercedes-benz',
   'bmw',
   'infiniti',
   'jaguar',
   'land_rover',
   'porsche',
   'lexus',
   'maserati'],
  'hajtaslanc': ['Hibrid', 'Elektromos', 'Dízel', 'Benzin']},
 'all': ['hany_eves',
  'hany_eves_2',
  'MEGTETT_KM',
  'MEGTETT_KM_2',
  'KLÍMA',
  'LÉGZSÁK',
  'ALUFELNI',
  'ASR',
  'AUTOMATA',
  'BI_XENON',
  'BLUETOOTH',
  'LOERO',
  'audi',
  'mercedes-benz',
  'bmw',
  'infiniti',
  'jaguar',
  'land_rover',
  'porsche',
  'lexus',
  'maserati',
  'Hibrid',
  'Elektromos',
  'Dízel',
  'Benzin'],
 'boolen': ['KLÍMA',
  'LÉGZSÁK',
  'ALUFELNI',
  'ASR',
  'AUTOMATA',
  'BI_XENON',
  'BLUETOOTH'],
 'number': ['hany_eves', 'hany_eves_2', 'MEGTETT_KM', 'MEGTETT_KM_2', 'LOERO']}

In [342]:
with open('data/dict.json') as json_file:  
    data_dict = json.load(json_file)

In [344]:
numbers  = [val for idx,val in enumerate(data_dict["number"]) if data_dict["number"][idx][:5]!=data_dict["number"][idx-1][:5]]

In [346]:
list(data_dict["Dropdown"].keys())


['auto_marka', 'hajtaslanc']